In [7]:
!pip install transformers datasets torch streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
dataset = load_dataset("imdb")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
from transformers import TrainingArguments, Trainer
from tqdm.auto import tqdm

# Custom blue progress bar style
tqdm.pandas(bar_format="{l_bar}{bar:30}{r_bar}", colour="blue")

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,   # adjust as needed
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"   # 🚫 disable wandb / tensorboard / mlflow
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=dataset["test"].shuffle(seed=42).select(range(500)),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=750, training_loss=0.28831404622395834, metrics={'train_runtime': 6706.5983, 'train_samples_per_second': 0.895, 'train_steps_per_second': 0.112, 'total_flos': 394666583040000.0, 'train_loss': 0.28831404622395834, 'epoch': 3.0})

In [ ]:

# Manually evaluate after training
results = trainer.evaluate()
print("📊 Evaluation Results:", results)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


📊 Evaluation Results: {'eval_loss': 0.6006534695625305, 'eval_accuracy': 0.86, 'eval_f1': 0.86, 'eval_precision': 0.8464566929133859, 'eval_recall': 0.8739837398373984, 'eval_runtime': 160.6552, 'eval_samples_per_second': 3.112, 'eval_steps_per_second': 0.392, 'epoch': 3.0}


In [ ]:
trainer.save_model("bert-sentiment")
tokenizer.save_pretrained("bert-sentiment")


('bert-sentiment/tokenizer_config.json',
 'bert-sentiment/special_tokens_map.json',
 'bert-sentiment/vocab.txt',
 'bert-sentiment/added_tokens.json')

In [ ]:
from google.colab import files
import shutil

# Zip the fine-tuned model
shutil.make_archive("bert-sentiment", 'zip', "bert-sentiment")

# Download the zip
files.download("bert-sentiment.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Upload zip back when needed
!unzip -o bert-sentiment.zip


unzip:  cannot find or open bert-sentiment.zip, bert-sentiment.zip.zip or bert-sentiment.zip.ZIP.


In [1]:
# ********** To use when runtime is disconnected **********

from google.colab import files
uploaded = files.upload()  # choose bert-sentiment.zip from your PC
!mkdir -p bert-sentiment
!unzip -o bert-sentiment.zip -d bert-sentiment

Saving bert-sentiment.zip to bert-sentiment.zip
Archive:  bert-sentiment.zip
  inflating: bert-sentiment/special_tokens_map.json  
  inflating: bert-sentiment/training_args.bin  
  inflating: bert-sentiment/tokenizer_config.json  
  inflating: bert-sentiment/vocab.txt  
  inflating: bert-sentiment/model.safetensors  
  inflating: bert-sentiment/config.json  


In [2]:
!ls bert-sentiment


config.json	   special_tokens_map.json  training_args.bin
model.safetensors  tokenizer_config.json    vocab.txt


In [3]:
%%writefile app.py
import streamlit as st
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import os

# ----------- Streamlit Page Config -----------
st.set_page_config(page_title="🎬 Movie Review Sentiment Analysis", layout="wide")

# ----------- Model Path Handling -----------
MODEL_PATH = "./bert-sentiment"  # folder must exist after unzip

if not os.path.exists(MODEL_PATH) or len(os.listdir(MODEL_PATH)) == 0:
    st.error("❌ Model folder 'bert-sentiment' not found or empty. Please train or upload 'bert-sentiment.zip' and unzip first.")
else:
    try:
        tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
        model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
        sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

        # ----------- Custom CSS -----------
        st.markdown("""
        <style>
        /* Background Animation */
        @keyframes gradient {
            0% {background-position: 0% 50%;}
            50% {background-position: 100% 50%;}
            100% {background-position: 0% 50%;}
        }
        .stApp {
            background: linear-gradient(-45deg, #141E30, #243B55, #232526, #414345);
            background-size: 400% 400%;
            animation: gradient 15s ease infinite;
            color: white;
        }
        h1 {
            text-align: center;
            font-size: 2.8em;
            margin-bottom: 0.3em;
            color: #FFD700;
            text-shadow: 2px 2px 5px black;
        }
        .subtitle {
            text-align: center;
            font-size: 1.2em;
            margin-bottom: 20px;
            color: #E0E0E0;
        }
        textarea {
            border-radius: 12px !important;
            border: 2px solid #FFD700 !important;
            background: rgba(255,255,255,0.1) !important;
            color: white !important;
            font-size: 1.1em !important;
        }
        .stButton>button {
            background: linear-gradient(135deg, #FFD700, #FFA500);
            color: black;
            font-weight: bold;
            border-radius: 12px;
            padding: 10px 20px;
            transition: 0.3s;
            border: none;
        }
        .stButton>button:hover {
            background: linear-gradient(135deg, #FFA500, #FF4500);
            color: white;
            transform: scale(1.05);
        }
        .result-box {
            background: rgba(0,0,0,0.6);
            padding: 20px;
            border-radius: 12px;
            margin-top: 25px;
            text-align: center;
            font-size: 1.3em;
            box-shadow: 0px 0px 15px rgba(255,215,0,0.5);
        }
        .positive {color: #00FF7F; font-weight: bold;}
        .negative {color: #FF6347; font-weight: bold;}
        .divider {
            text-align: center;
            margin: 20px 0;
            font-size: 1.5em;
            color: #FFD700;
        }
        </style>
        """, unsafe_allow_html=True)

        # ----------- App UI -----------
        st.title("🎬 IMDB Movie Review Sentiment Classifier")
        st.markdown("<p class='subtitle'>Enter a movie review below and discover if it’s a 👍 Positive hit or a 👎 Negative flop!</p>", unsafe_allow_html=True)

        st.markdown("<div class='divider'>🎞️ ——————————— 🎞️</div>", unsafe_allow_html=True)

        review = st.text_area("✍️ Write your movie review:", height=150)

        if st.button("🔍 Analyze Sentiment"):
            if review.strip() == "":
                st.warning("⚠️ Please enter a review before analyzing.")
            else:
                with st.spinner("🤔 Analyzing the review..."):
                    try:
                        result = sentiment_pipeline(review)[0]
                        label = result["label"]
                        score = result["score"]

                        # Map HuggingFace labels to human readable
                        if label == "LABEL_1":
                            sentiment = "👍 Positive"
                            css_class = "positive"
                        else:
                            sentiment = "👎 Negative"
                            css_class = "negative"

                        st.markdown(f"""
                        <div class="result-box">
                            <h3>🧾 Sentiment Result:</h3>
                            <p class="{css_class}">{sentiment}</p>
                            <p>Confidence: <b>{score:.2f}</b></p>
                        </div>
                        """, unsafe_allow_html=True)
                    except Exception as e:
                        st.error(f"⚠️ Error during analysis: {str(e)}")
    except Exception as e:
        st.error(f"⚠️ Failed to load model: {str(e)}")


Writing app.py


In [4]:
!kill -9 $(ps -A | grep ngrok | awk '{print $1}')


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [8]:
# Kill previous streamlit if running
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No previous Streamlit instance"

import time
from pyngrok import ngrok
import subprocess

# 🔑 Auth token
ngrok.set_auth_token("2ztgrogNy3iJnxaz8wchWgCw1X7_4vgQZE6DfL2kHMu2DmQzM")

# Start Streamlit in background (suppress logs so Python can continue)
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT
)

time.sleep(5)  # wait for streamlit to boot

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print("🌐 Your public ngrok URL:", public_url.public_url)


No previous Streamlit instance
🌐 Your public ngrok URL: https://cdf8a2114277.ngrok-free.app
